In [15]:
import pandas as pd
import re

import _codes
import _util
import _config
#re
pattern = re.compile(r'\d/4')

#file_path setting
files = _util.get_files(_config.hospital_read_path)

df_list = []
for file in files:
    df_list.append(pd.read_csv(_config.hospital_read_path + '\\' + file, encoding=_config.euc_kr))
df = pd.concat(df_list, ignore_index=False)

_cols = {}
for col in df.columns:
    val = str(col)
    if col.count('20') > 0 and col.count('.1/4') > 0: val = val.replace('.1/4', '')
    elif col.count('시군구') > 0: val = 'sgg_nm'
    elif col.count('Unnamed') > 0: continue
    elif col.count('.2/4') > 0: continue
    elif col.count('.3/4') > 0: continue
    elif col.count('.4/4') > 0: continue

    _cols[col] = val

df.rename(columns=_cols, inplace=True)
df = df[list(_cols.values())]
df = pd.DataFrame(df)

num_cols = [col for col in df.columns if col.isnumeric()]
not_num_cols = [col for col in df.columns if not col.isnumeric()]

df = df.melt(
    id_vars=not_num_cols
    , value_vars=num_cols
    , var_name='year'
    , value_name='개수'
    , ignore_index=False
).dropna()
df.reset_index(drop=False, inplace=True)
df.to_csv(_config.hospital_write_path + '\\' + 'hospital_2' + '.csv', encoding=_config.euc_kr, index=False)

sido_cd = None
for idx, row in df.iterrows():
    sgg_nm = str(row['sgg_nm']).strip()

    if sgg_nm == '고양시 일산 동구': 
        sgg_nm = '고양시 일산동구'
        df.loc[idx, 'sgg_nm'] = sgg_nm
    if sgg_nm == '고양시 일산 서구': 
        sgg_nm = '고양시 일산서구'
        df.loc[idx, 'sgg_nm'] = sgg_nm

    sgg_split = sgg_nm.split(' ')
    if len(sgg_split) > 1: 
        sgg_nm = sgg_split[1]
        # print(sgg)

    #year
    year = str(row['year']).strip()

    #sido_cd mapping
    sido_cd = _util.sido_cd_map.get(sgg_nm, sido_cd)

    #sgg_cd mapping2
    case_no = 1
    sgg_cd = _util.getSggCd(sido_cd, sgg_nm, year, 1)
    df.loc[idx, 'sgg_cd'] = f'`{str(sgg_cd)}'
    df.loc[idx, 'sido_cd'] = f'`{sido_cd}'

print(df)

df = df.pivot_table(
    index=['sido_cd', 'sgg_cd', 'sgg_nm', 'year']
    , columns='요양기관종별'
    , values='개수'
    , aggfunc='sum'
)
df.reset_index(drop=False, inplace=True)

# _list = []
# for idx, row in df.iterrows():
#     cnt = row['병원']
    
#     import math
#     if math.isnan(cnt): continue
#     elif cnt == 0: continue

#     _list.append(row)

# df = pd.DataFrame(_list)
# df.reset_index(drop=False, inplace=False)

_list = []
for col in df.columns:
    if col.count('계') > 0: continue
    _list.append(col)
df = df[_list]

df.to_csv(_config.hospital_write_path + '\\' + 'hospital' + '.csv', encoding=_config.euc_kr, index=False)
print("end write file")


       index sgg_nm  요양기관종별          항목 단위  year       개수  sgg_cd sido_cd
0          0     전체       계  요양기관 현황[개]  개  2009  78829.0     `00     `00
1          1     전체  상급종합병원  요양기관 현황[개]  개  2009     44.0     `00     `00
2          2     전체    종합병원  요양기관 현황[개]  개  2009    269.0     `00     `00
3          3     전체      병원  요양기관 현황[개]  개  2009   1211.0     `00     `00
4          4     전체    요양병원  요양기관 현황[개]  개  2009    714.0     `00     `00
...      ...    ...     ...         ... ..   ...      ...     ...     ...
65087   4632    제주시   보건의료원  요양기관 현황[개]  개  2023      0.0  `39010     `39
65088   4633    제주시     보건소  요양기관 현황[개]  개  2023      3.0  `39010     `39
65089   4634    제주시    보건지소  요양기관 현황[개]  개  2023      6.0  `39010     `39
65090   4635    제주시   보건진료소  요양기관 현황[개]  개  2023     23.0  `39010     `39
65091   4636    제주시      약국  요양기관 현황[개]  개  2023    240.0  `39010     `39

[65092 rows x 9 columns]
end write file


In [16]:
import _config
import _util
import _codes
import pandas as pd
import math

def create_table(columns):
    columns = list(columns)
    for idx in range(len(columns)):
        columns[idx] = f'"{columns[idx]}"'

        if str(columns[idx]).count('sido_cd') > 0: columns[idx] = columns[idx] + ' varchar(2)'
        elif str(columns[idx]).count('sgg_cd') > 0: columns[idx] = columns[idx] + ' varchar(6)'
        elif str(columns[idx]).count('sgg_nm') > 0: columns[idx] = columns[idx] + ' varchar(20)'
        elif str(columns[idx]).count('year') > 0: columns[idx] = columns[idx] + ' varchar(4)'
        else: columns[idx] = columns[idx] + ' int4'

    sql = f'''
        do $$
        begin
            if exists (select 1 from pg_tables where tablename = 'hospital') then
            drop table hospital cascade;
            end if;
            if not exists (select 1 from pg_tables where tablename = 'hospital') then
            create table if not exists hospital(
            {', '.join(columns)}
            );
            end if;
        end $$;
    '''

    _util.execute_sql(sql)

def insert_data(df, columns):
    columns = list(columns)
    for idx in range(len(columns)):
        columns[idx] = f'"{columns[idx]}"'

    df = pd.DataFrame(df)
    data = df.values.tolist()

    for chunk in _util.chunker(data, 1000):
        values = []
        for row in chunk:
            for idx in range(len(columns)):
                if str(row[idx]).count('`') > 0:
                    val = str(row[idx]).replace('`', '')
                    row[idx] = f'\'{val}\''
                elif isinstance(row[idx], (int, float)): row[idx] = _util.objToInt(row[idx])
                else: row[idx] = f'\'{str(row[idx])}\''
                    
                if row[idx] == 'None': row[idx] = '\'\''

            row = [str(item) for item in row]
            values.append(f"({', '.join(row)})")

        sql = f'''
            insert into hospital(
                {','.join(columns)})
            values {', '.join(values)}
        '''
        _util.execute_sql(sql)

file_name = 'hospital' + '.csv'
file_path = _config.hospital_write_path + '\\' + file_name

df = pd.read_csv(file_path, encoding=_config.euc_kr)
create_table(df.columns)
insert_data(df, df.columns)



        do $$
        begin
            if exists (select 1 from pg_tables where tablename = 'hospital') then
            drop table hospital cascade;
            end if;
            if not exists (select 1 from pg_tables where tablename = 'hospital') then
            create table if not exists hospital(
            "sido_cd" varchar(2), "sgg_cd" varchar(6), "sgg_nm" varchar(20), "year" varchar(4), "병원" int4, "보건소" int4, "보건의료원" int4, "보건지소" int4, "보건진료소" int4, "상급종합병원" int4, "약국" int4, "요양병원" int4, "의원" int4, "정신병원" int4, "조산원" int4, "종합병원" int4, "치과병원" int4, "치과의원" int4, "한방병원" int4, "한의원" int4
            );
            end if;
        end $$;
    

            insert into hospital(
                "sido_cd","sgg_cd","sgg_nm","year","병원","보건소","보건의료원","보건지소","보건진료소","상급종합병원","약국","요양병원","의원","정신병원","조산원","종합병원","치과병원","치과의원","한방병원","한의원")
            values ('00', '00', '전체', 2009, 1211, 239, 17, 1286, 1912, 44, 20880, 714, 26640, 0, 51, 269, 172, 13831, 139, 11424), ('00', '00', '

In [19]:
import _config
import _util
import _codes

def update():

    sql = f'''
        select sido_cd, sgg_cd, sgg_nm, year
        from hospital
        where sgg_cd = 'None'
    '''

    rows = _util.execute_sql(sql)

    if rows is None: return

    _rows = []
    for row in rows:
        sido_cd, sgg_cd, sgg_nm, year = row

        sgg_cd = _util.getSggCd(sido_cd, sgg_nm, year, 2)

        # if sgg_cd is None: print(sgg_cd, sgg_nm)
        row = sido_cd, sgg_cd, sgg_nm, year
        _rows.append(row)

    for row in _rows:
        sido_cd, sgg_cd, sgg_nm, year = row

        if str(sgg_nm).count('군위군') > 0 and str(sgg_cd).count('None') > 0: sgg_cd = '22520'
        if str(sgg_nm).count('군위군') > 0 and sgg_cd is None > 0: sgg_cd = '22520'
        sql = f'''
            update hospital
            set sgg_cd = \'{sgg_cd}\'
            where sido_cd = \'{sido_cd}\' and sgg_nm = \'{sgg_nm}\' and year = \'{year}\'
        '''

        _util.execute_sql(sql)
        if sgg_cd == '22520':
            sql = f'''
                update hospital
                set sido_cd = '22'
                where sgg_cd = \'{sgg_cd}\'
            '''
            _util.execute_sql(sql)

update()


        select sido_cd, sgg_cd, sgg_nm, year
        from hospital
        where sgg_cd = 'None'
    
[('29', 'None', '세종시', '2014'), ('29', 'None', '세종시', '2015'), ('29', 'None', '세종시', '2016'), ('29', 'None', '세종시', '2017'), ('29', 'None', '세종시', '2018'), ('29', 'None', '세종시', '2019'), ('31', 'None', '여주시', '2010'), ('31', 'None', '여주시', '2011'), ('31', 'None', '여주시', '2012'), ('31', 'None', '여주시', '2013'), ('31', 'None', '여주시', '2014'), ('32', 'None', '강원', '2010'), ('32', 'None', '강원', '2011'), ('32', 'None', '강원', '2012'), ('32', 'None', '강원', '2013'), ('32', 'None', '강원', '2014'), ('32', 'None', '강원', '2015'), ('32', 'None', '강원', '2016'), ('32', 'None', '강원', '2017'), ('32', 'None', '강원', '2018'), ('32', 'None', '강원', '2019'), ('32', 'None', '강원', '2021'), ('32', 'None', '강원', '2022'), ('32', 'None', '강원', '2023'), ('34', 'None', '당진시', '2010'), ('34', 'None', '당진시', '2011'), ('34', 'None', '당진시', '2012'), ('34', 'None', '당진시', '2013'), ('34', 'None', '천안시 서북구', '2009'), ('35',

In [13]:
# 특정년도 병의원 데이터 가져오기

import _util
import _config
import pandas as pd

def dataToCsv(year):
    file_nm = f'hospital_{year}'
    sql =f'''
    --구제시
        do $$
        begin
            if exists (select 1 from pg_matviews where matviewname = 'v_gujesi_hospital_{year}_5179') then
            drop materialized view v_gujesi_hospital_{year}_5179 cascade;
            end if;
            create materialized view v_gujesi_hospital_{year}_5179 as
            with gu as (
                select *
                from gujaesi_sgg_{year}_5179 gs 
            )
            , hos as (
                select *
                from hospital h
                where "year" = {year}::text
            )
            select row_number() over() as fid
            , coalesce(h.sido_cd, substring(g.sgg_cd, 1, 2)) as sido_cd, g.sgg_cd, g.sgg_nm
            , sum(h."의원") as "의원", sum(h."병원") as "병원", sum(h."상급종합병원") as "상급종합병원"
            , sum(h."보건소") as "보건소", sum(h."보건지소") as "보건지소", sum(h."보건진료소") as "보건진료소", sum(h."보건의료원") as "보건의료원"
            , sum(h."치과의원") as "치과의원", sum(h."치과병원") as "치과병원", sum(h."한의원") as "한의원", sum(h."한방병원") as "한방병원", sum(h."조산원") as "조산원"
            , sum(h."요양병원") as "요양병원", sum(h."정신병원") as "정신병원", sum(h."약국") as "약국"
            , (select geom from gu where sgg_cd = g.sgg_cd) as geom
            from gu g
            left join hos h on substring(h.sgg_cd, 1, 4) = substring(g.sgg_cd, 1, 4)
            group by h.sido_cd, g.sgg_cd , g.sgg_nm;
        end $$;

        --구제시 아님
        do $$
        begin
            if exists (select 1 from pg_matviews where matviewname = 'v_nogujesi_hospital_{year}_5179') then
            drop materialized view v_nogujesi_hospital_{year}_5179 cascade;
            end if;
            create materialized view v_nogujesi_hospital_{year}_5179 as
            with ngu as (
                select *
                from nogujaesi_sgg_{year}_5179 gs 
            )
            , hos as (
                select *
                from hospital h
                where "year" = {year}::text
            )
            select row_number() over() as fid
            , coalesce(h.sido_cd, substring(g.sgg_cd, 1, 2)) as sido_cd, g.sgg_cd , g.sgg_nm
            , sum(h."의원") as "의원", sum(h."병원") as "병원", sum(h."상급종합병원") as "상급종합병원"
            , sum(h."보건소") as "보건소", sum(h."보건지소") as "보건지소", sum(h."보건진료소") as "보건진료소", sum(h."보건의료원") as "보건의료원"
            , sum(h."치과의원") as "치과의원", sum(h."치과병원") as "치과병원", sum(h."한의원") as "한의원", sum(h."한방병원") as "한방병원", sum(h."조산원") as "조산원"
            , sum(h."요양병원") as "요양병원", sum(h."정신병원") as "정신병원", sum(h."약국") as "약국"
            , g.geom
            from ngu g
            left join hos h on h.sgg_cd = g.sgg_cd
            group by h.sido_cd, g.sgg_cd , g.sgg_nm, g.geom;
        end $$;

        do $$
        begin
            if exists (select 1 from pg_matviews where matviewname = 'v_hospital_{year}_5179') then
            drop materialized view v_hospital_{year}_5179 cascade;
            end if;
            create materialized view v_hospital_{year}_5179 as
            select row_number() over() as fid, sido_cd, sgg_cd, sgg_nm
                , "의원", "병원", "상급종합병원"
                , "보건소", "보건지소", "보건진료소", "보건의료원"
                , "치과의원", "치과병원", "한의원", "한방병원", "조산원"
                , "요양병원", "정신병원", "약국"
            from (
                select *
                from v_gujesi_hospital_{year}_5179
                union all 
                select *
                from v_nogujesi_hospital_{year}_5179
            );
        end $$;
    '''
    _util.execute_sql(sql)

    # _cols = _util.getMatViewCols(f'v_hospital_{year}_5179')
    # if _cols is None: print(_cols)

    sql = f'''
        select sido_cd, sgg_cd, sgg_nm
        , "의원", "병원", "상급종합병원"
        , "보건소", "보건지소", "보건진료소", "보건의료원"
        , "치과의원", "치과병원", "한의원", "한방병원", "조산원"
        , "요양병원", "정신병원", "약국"
        from v_hospital_{year}_5179
    '''
    rows = _util.execute_sql(sql)
    if rows is None: return
    _list = []

    for row in rows:
        _list.append({
            'sido_cd': row[0]
            , 'sgg_cd': row[1]
            , '"의원"': row[2]
            , '"병원"': row[3]
            , '"상급종합병원"': row[4]
            , '"보건소"': row[5]
            , '"보건지소"': row[6]
            , '"보건진료소"': row[7]
            , '"보건의료원"': row[8]
            , '"치과의원"': row[9]
            , '"치과병원"': row[10]
            , '"한의원"': row[11]
            , '"한방병원"': row[12]
            , '"조산원"': row[13]
            , '"요양병원"': row[14]
            , '"정신병원"': row[15]
            , '"약국"': row[16]
        })

    df = pd.DataFrame(_list)
    df.to_csv(_config.hospital_write_path + '\\' + file_nm + '.csv', encoding=_config.euc_kr)

for year in _config.layer_years: dataToCsv(year)


    --구제시
        do $$
        begin
            if exists (select 1 from pg_matviews where matviewname = 'v_gujesi_hospital_2000_5179') then
            drop materialized view v_gujesi_hospital_2000_5179 cascade;
            end if;
            create materialized view v_gujesi_hospital_2000_5179 as
            with gu as (
                select *
                from gujaesi_sgg_2000_5179 gs 
            )
            , hos as (
                select *
                from hospital h
                where "year" = 2000::text
            )
            select row_number() over() as fid
            , coalesce(h.sido_cd, substring(g.sgg_cd, 1, 2)) as sido_cd, g.sgg_cd, g.sgg_nm
            , sum(h."의원") as "의원", sum(h."병원") as "병원", sum(h."상급종합병원") as "상급종합병원"
            , sum(h."보건소") as "보건소", sum(h."보건지소") as "보건지소", sum(h."보건진료소") as "보건진료소", sum(h."보건의료원") as "보건의료원"
            , sum(h."치과의원") as "치과의원", sum(h."치과병원") as "치과병원", sum(h."한의원") as "한의원", sum(h."한방병원") as "한방병원", 